## Geographical plots

In this part of the hands-on we will be plotting information on geographical. This goes by the name of Geographic Information Systems (GIS), which, as the name implies, is some sort of information visualization.

GIS comes in many forms but for this example we will be making a **Choropleth**. A Choropleth is a map visualisation where parts of a map are shaded in proportion to a data measurement or statistical value of choice. 

For this example we are going to visualise the results of the election of the Ukraine referendum for each municipality in the Netherlands. For this we will first need to import Pandas and prepare our data.

In [ ]:
# First lets clear all previous python imports and variables by resetting the python kernel.
%reset -f

import pandas

results = pandas.read_excel('./data/geographical/referendum uitslag per gemeente.xlsx')

# Rename several column names for beter reference and to translate them to English
results = results.rename_axis({'Gemeente':'municipality','geldige stemmen':'valid', 'ongeldig':'invalid', 
                               'Tegen':'against', 'Voor':'infavor', 'kiesgerechtigden':'eligible',
                               'blanco':'blank'}, 
                                  axis='columns')
# Remove rows with totals (The NaN entry)
results = results.dropna(subset=['municipality'])

# Add turnout column
results['turnout'] = results['valid'] + results['invalid']

# Rearange columns in more logical order
results = results[['municipality', 'eligible', 'turnout', 'valid', 'invalid', 'infavor', 'against', 'blank']]

# Create a percentage dataframe
p_results = pandas.DataFrame(dict(
    municipality = results['municipality'],
    p_turnout = 100.0 * results['turnout'] / results['eligible'],
    p_valid = 100.0 * results['valid'] / results['turnout'],
    p_invalid = 100.0 * results['invalid'] / results['turnout'],
    p_infavor = 100.0 * results['infavor'] / results['valid'],
    p_against = 100.0 * results['against'] / results['valid'],
    p_blank = 100.0 * results['blank'] / results['valid']
    ))

p_results

As you can see we now prepared our data and created a DataFrame that consists of the votes in favor, votes against, blank votes, invalid votes, valid votes and turnout in percentages. This way we can easily color code the municipalities and compare them to each other in terms of the data columns in the DataFrame. 

Now lets create the map. The first map we will create will be a choropleth displaying the percentage of votes in favor of in the Ukraine referendum.

In [ ]:
from IPython.display import display, IFrame
import folium

geo_path = r'./data/geographical/Gemeentegrenzen_2016_zonder_water_simplified_wgs84.geojson'

ref_map = folium.Map(
    location=[52.139177, 5.327108], # This will center the view on the world map where the Netherlands is located
    tiles='Mapbox Bright',          # This creates a base map and in this case its the Mapbox Bright basemap
    zoom_start=8)                   # This will zoom in on the center of view to get the Netherlands in full frame

ref_map.choropleth(
    # This is the path to the geojson file that contains all the municipality shapes and locations             
    geo_path=geo_path,                     
    # We will use the percentage dataframe for the choropleth mapping          
    data=p_results,                        
    # Municipality will be used for the mapping key and p_valid for its value
    columns=['municipality', 'p_infavor'], 
    # Use GM_NAAM (short for municipality name) as keys for colormapping 
    key_on='feature.properties.GM_NAAM',   
    # We are going to use a color map from yellow to green
    fill_color='YlGn',                     
    # This gives municipality shapes some opacity so that we can still see the background
    fill_opacity=0.7,                      
    # This gives the lines around the municipality shapes some opacity so that they don't stand out too much
    line_opacity=0.2,                      
    # The legend
    legend_name='In favor (%)')            

ref_map.save('./data/geographical/referendum.html')              # This will save the map in a HTML format

display(IFrame('./data/geographical/referendum.html', 800,800))  # This displays the map in an Iframe

If everything went according to plan you should see a map of the Netherlands with patches shaded green propotional to the *p_infavor* data values. From this choropleth you can already see that the percentage of voters in favor is relatively higher in and around the big cities. For reference you can compare your map to the map created by [NOS](http://app.nos.nl/datavisualisatie/referendum-2016/).

* * *
* * *

#### Exercise

For this exercise you are going to look at how the other data columns are represented across the Netherlands by changing the **columns** variable second entry, in the **ref_map.geo_json** function below, to one of the other data columns in our DataFrame. Also try to represent the results with the right color map (in personal preferences of course), from purple to blue (PuBu), from yellow to green (YlGn) or from orange to red (OrRd).

In [ ]:
colorMaps = ['PuBu', 'YlGn', 'OrRd']

geo_path = r'./data/geographical/Gemeentegrenzen_2016_zonder_water_simplified_wgs84.geojson'

ref_map = folium.Map(
    location=[52.139177, 5.327108], # This will center the view on the world map where the Netherlands is located
    tiles='Mapbox Bright',          # This creates a base map and in this case its the Mapbox Bright basemap  
    zoom_start=8)                   # This will zoom in on the center of view to get the Netherlands in full frame

ref_map.choropleth(
    # This is the path to the geojson file that contains all the municipality shapes and locations      
    geo_path=geo_path,                     
    # We will use the percentage dataframe for the choropleth mapping      
    data=p_results,                       
    # Municipality will be used for the mapping key and one of the other datacolumns as a value     
    columns=['municipality', <FILL IN>],  
    # Use GM_NAAM (short for municipality name) as keys for colormapping   
    key_on='feature.properties.GM_NAAM',   
    # We are going to use a color map stored in the colormap dictionary
    fill_color=<FILL IN>,      
    # This gives municipality shapes some opacity so that we can still see the background       
    fill_opacity=0.7,                      
    # This gives the lines around the municipality shapes some opacity so that they don't stand out too much     
    line_opacity=0.2,                 
    # The legend
    legend_name=<FILL IN>)            

ref_map.save('./data/geographical/referendum.html')              # This will save the map in a HTML format

display(IFrame('./data/geographical/referendum.html', 800,800))  # This displays the map in an Iframe

***
***

This chapter concludes the main visualation chapters of this course now you may proceed the conclusion notebook of this course for some take away messages: [Conclusion](./5. Conclusion.ipynb)